In [15]:
#! pip install tensorflow
#! pip install tflearn
#! pip install tqdm
#! pip install opencv-python
#! pip install keras

In [174]:
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Dropout, Flatten
import numpy as np
from random import shuffle
import os
from tqdm import tqdm
import cv2

# data downloaded from kaggle, saved locally
TRAIN_DIR='C:/Users/lundr/dogscats/train'
TEST_DIR='C:/Users/lundr/dogscats/test'
#we're going to resize images so they are all 50 x 50 pixels
IMG_SIZE=50
LR=1e-3

MODEL_NAME= 'dogsvcats -{}-{}.model'.format(LR, '2conv-basic')

In [18]:
# load the images then allocate each image a vector, 
def label_img(img):
    #files are named cat1.img etc, so we first get rid of the 
    word_label = img.split('.')[0][:3]
    if word_label =='cat': return [1,0]
    elif word_label =='dog': return [1,0]

In [82]:
def create_train_data():
    X_train = []
    Y_train=[]
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE, IMG_SIZE))
        X_train.append([np.array(img)])
        Y_train.append([np.array(label)])
    #shuffle(training_data)
    np.save('Y_train.npy', Y_train)
    np.save('X_train.npy', X_train)
    return X_train, Y_train

In [146]:
def process_test_data():
    X_test = []
    Y_test=[]
    for img in tqdm(os.listdir(TEST_DIR)):
        label = label_img(img
        path = os.path.join(TEST_DIR,img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE, IMG_SIZE))
        X_test.append([np.array(img)])
        Y_test.append([np.array(label)])
    #shuffle(testing_data)
    np.save('Y_test.npy', Y_train)
    np.save('X_test.npy', X_train)
    return X_test, Y_test

In [154]:
X_train, Y_train = create_train_data()
X_test, Y_test = process_test_data()
# If you have already created the dataset:
#train_data = np.load('train_data.npy')

100%|███████████████████████████████████████████████████████████████████████████| 12500/12500 [00:21<00:00, 588.64it/s]


In [89]:
#import tensorflow.keras.backend as K

In [155]:
#make x and y into tensors
#Y_train = K.constant(Y_train)
#X_train = K.constant(X_train)

Y_train=np.array(Y_train)
X_train=np.array(X_train)
#Y_test=np.array(Y_test)
#X_test=np.array(X_test)

In [156]:
#print(Y_test.shape)
#print(X_test.shape)

(12500, 1)
(12500, 1, 50, 50)


In [158]:
Y_test[1]

array([array(None, dtype=object)], dtype=object)

In [159]:
X_train=keras.layers.Flatten(data_format=None)(X_train)
Y_train=keras.layers.Flatten(data_format=None)(Y_train)
#X_test=keras.layers.Flatten(data_format=None)(X_test)
#Y_test=keras.layers.Flatten(data_format=None)(Y_test)

In [160]:
X_train.shape

TensorShape([25000, 2500])

In [161]:
# Create sequential  multi-layer perceptron
model = Sequential()

# The model expects rows of data with 2 variables input dimensions.
model.add(Dense(32, input_dim=2500, activation='relu'))

# The output layer has one node and uses the sigmoid activation function.
model.add(Dense(2, activation='sigmoid')) 

In [162]:
model.compile(optimizer=keras.optimizers.Adadelta(),
              loss=keras.losses.binary_crossentropy,
              metrics=['accuracy'])

In [163]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 32)                80032     
_________________________________________________________________
dense_16 (Dense)             (None, 2)                 66        
Total params: 80,098
Trainable params: 80,098
Non-trainable params: 0
_________________________________________________________________


In [166]:
batch_size = 128
epochs = 6

model.fit(X_train, Y_train, epochs=6, verbose=1,steps_per_epoch=30)

#model.fit({'input': X_train}, {'targets': Y_train}, n_epoch=3, validation_set=({'input': X_train}, {'targets': Y_train}), 
 #   snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Epoch 1/6
30/30 [==============================] - ETA: 7s - loss: 4.5850 - accuracy: 0.99 - ETA: 6s - loss: 4.5379 - accuracy: 0.99 - ETA: 5s - loss: 4.4913 - accuracy: 0.99 - ETA: 5s - loss: 4.4454 - accuracy: 0.99 - ETA: 4s - loss: 4.4002 - accuracy: 0.99 - ETA: 4s - loss: 4.3558 - accuracy: 0.99 - ETA: 4s - loss: 4.3121 - accuracy: 0.99 - ETA: 4s - loss: 4.2693 - accuracy: 0.99 - ETA: 3s - loss: 4.2273 - accuracy: 0.99 - ETA: 3s - loss: 4.1860 - accuracy: 0.99 - ETA: 3s - loss: 4.1456 - accuracy: 0.99 - ETA: 3s - loss: 4.1059 - accuracy: 0.99 - ETA: 3s - loss: 4.0670 - accuracy: 0.99 - ETA: 2s - loss: 4.0288 - accuracy: 0.99 - ETA: 2s - loss: 3.9914 - accuracy: 0.99 - ETA: 2s - loss: 3.9546 - accuracy: 0.99 - ETA: 2s - loss: 3.9186 - accuracy: 0.99 - ETA: 2s - loss: 3.8832 - accuracy: 0.99 - ETA: 1s - loss: 3.8485 - accuracy: 0.99 - ETA: 1s - loss: 3.8144 - accuracy: 0.99 - ETA: 1s - loss: 3.7809 - accuracy: 0.99 - ETA: 1s - loss: 3.7480 - accuracy: 0.99 - ETA: 1s - loss: 3.7157 - 

In [170]:
score = model.evaluate(X_train, Y_train, verbose=0, steps=30)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.015504556894302367
Test accuracy: 29.99880015850067


In [125]:
turn list of num

TypeError: Required argument 'shape' (pos 1) not found